## Q1. Running Elastic

In [1]:
!curl localhost:9200

{
  "name" : "0be358a9f843",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "hhOf7kdsToqyQoNsbXoCWg",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


## Getting the data

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
from elasticsearch import Elasticsearch

In [4]:
es_client = Elasticsearch('http://localhost:9200') 

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [6]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:22<00:00, 42.79it/s]


### Which function do you use for adding your data to elastic?
We use index

## Q3. Searching

Use only question and text fields and give question a boost of 4, and use "type": "best_fields".

In [8]:
def elastic_search(query, filter_course = None, size = 5):
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }
    if filter_course:
        search_query["query"]["bool"].update({"filter": {
                    "term": {
                        "course": filter_course
                    }
                }})

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    scores = []
    
    for hit in response['hits']['hits']:
        scores.append(hit["_score"])
        result_docs.append(hit['_source'])
    
    return result_docs,scores

In [9]:
query = "How do I execute a command in a running docker container?"

In [10]:
elastic_search(query)

([{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'},
  {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for root:\nServer: PostgreSQL 16.1 (

### What's the score for the top ranking result?
84.050095

## Q4. Filtering
Now let's only limit the questions to machine-learning-zoomcamp.
Return 3 results. What's the 3rd question returned by the search engine?

In [11]:
query = "How do I execute a command in a running docker container?"

In [12]:
search_results , _ = elastic_search(query, "machine-learning-zoomcamp", 3)

In [13]:
search_results[2]["question"]

'How do I copy files from a different folder into docker container’s working directory?'

## Q5. Building a prompt

In [14]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [15]:
prompt = build_prompt(query, search_results)

In [16]:
len(prompt)

1462

## Q6. Tokens

In [17]:
%%capture
!pip install tiktoken

In [18]:
import tiktoken

In [19]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [21]:
len(encoding.encode(prompt))

322